In [2]:
%%bash
set -e

sudo apt-get update
sudo apt-get install -y wget apt-transport-https gnupg lsb-release

wget -qO - https://aquasecurity.github.io/trivy-repo/deb/public.key | sudo apt-key add -
echo "deb https://aquasecurity.github.io/trivy-repo/deb $(lsb_release -sc) main" \
  | sudo tee /etc/apt/sources.list.d/trivy.list

sudo apt-get update
sudo apt-get install -y trivy

trivy --version


Hit:1 https://packages.cloud.google.com/apt gcsfuse-bullseye InRelease
Hit:2 https://download.docker.com/linux/debian bullseye InRelease
Hit:3 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:4 https://deb.debian.org/debian bullseye InRelease
Hit:5 https://packages.cloud.google.com/apt google-compute-engine-bullseye-stable InRelease
Get:6 https://deb.debian.org/debian-security bullseye-security InRelease [27.2 kB]
Hit:7 https://deb.debian.org/debian bullseye-updates InRelease
Hit:8 https://packages.cloud.google.com/apt cloud-sdk-bullseye InRelease
Hit:9 https://packages.cloud.google.com/apt google-fast-socket InRelease
Fetched 27.2 kB in 1s (21.8 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
apt-transport-https is already the newest version (2.2.4).
gnupg is already the newest version (2.2.27-2+deb11u2).
gnupg set to manually installed.
lsb-release is already the newest version (11.1.0).
lsb-

OK
deb https://aquasecurity.github.io/trivy-repo/deb bullseye main
Hit:1 https://download.docker.com/linux/debian bullseye InRelease
Hit:2 https://deb.debian.org/debian bullseye InRelease
Hit:3 https://packages.cloud.google.com/apt gcsfuse-bullseye InRelease
Hit:4 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:5 https://deb.debian.org/debian-security bullseye-security InRelease
Hit:6 https://deb.debian.org/debian bullseye-updates InRelease
Hit:7 https://packages.cloud.google.com/apt google-compute-engine-bullseye-stable InRelease
Get:8 https://aquasecurity.github.io/trivy-repo/deb bullseye InRelease [3064 B]
Hit:9 https://packages.cloud.google.com/apt cloud-sdk-bullseye InRelease
Hit:10 https://packages.cloud.google.com/apt google-fast-socket InRelease
Get:11 https://aquasecurity.github.io/trivy-repo/deb bullseye/main amd64 Packages [370 B]
Fetched 3434 B in 1s (2512 B/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading

dpkg-preconfigure: unable to re-open stdin: No such file or directory


Fetched 39.3 MB in 1s (34.0 MB/s)
Selecting previously unselected package trivy.
(Reading database ... 145494 files and directories currently installed.)
Preparing to unpack .../trivy_0.54.1_amd64.deb ...
Unpacking trivy (0.54.1) ...
Setting up trivy (0.54.1) ...
Version: 0.54.1


In [ ]:
import json
import subprocess
import shutil
import requests
from google.auth import default
from google.auth.transport.requests import Request

IMAGE = "nginx:latest"
LOCATION = "us-central1"

# -------------------------
# Pre-flight checks
# -------------------------
if not shutil.which("trivy"):
    raise RuntimeError("Trivy is not installed on this Workbench VM")

# -------------------------
# Run Trivy scan
# -------------------------
print(f"🐳 Scanning image: {IMAGE}")

subprocess.run([
    "trivy", "image",
    "--scanners", "vuln",
    "--severity", "CRITICAL,HIGH,MEDIUM",
    "--format", "json",
    "--output", "trivy-report.json",
    IMAGE
], check=True)

# -------------------------
# Parse Trivy results
# -------------------------
with open("trivy-report.json") as f:
    report = json.load(f)

def count(sev):
    return sum(
        1 for r in report.get("Results", [])
        for v in (r.get("Vulnerabilities") or [])
        if v.get("Severity") == sev
    )

critical = count("CRITICAL")
high = count("HIGH")
medium = count("MEDIUM")

print(f"📊 Critical={critical}, High={high}, Medium={medium}")

# -------------------------
# Prompt Engineering
# -------------------------
PROMPT = f"""
<YOU ARE THE PROMPT ENGINEER
"""

# -------------------------
# Get ADC token
# -------------------------
credentials, project_id = default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
credentials.refresh(Request())
token = credentials.token

# -------------------------
# Vertex AI Gemini 2.5 Flash call
# -------------------------
url = (
    f"https://{LOCATION}-aiplatform.googleapis.com/v1/"
    f"projects/{project_id}/locations/{LOCATION}/"
    f"publishers/google/models/gemini-2.5-flash:generateContent"
)

payload = {
    "contents": [
        {
            "role": "user",
            "parts": [{"text": PROMPT}]
        }
    ]
}

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

print("🤖 Calling Gemini 2.5 Flash via Vertex AI...")

response = requests.post(url, headers=headers, json=payload)
response.raise_for_status()

data = response.json()
print("🔎 Raw Gemini response:")
print(json.dumps(data, indent=2))

# -------------------------
# Parse Gemini decision
# -------------------------
text = data["candidates"][0]["content"]["parts"][0]["text"]
decision_json = json.loads(text)

decision = decision_json["decision"]
risk = decision_json["risk_level"]
summary = decision_json["summary"]

print(f"🧠 Decision : {decision}")
print(f"⚠️  Risk     : {risk}")
print(f"📝 Summary  : {summary}")

if decision == "DEPLOY":
    print("✅ Image approved for deployment")
else:
    print("❌ Image rejected for deployment")


🐳 Scanning image: nginx:latest


2025-12-20T01:33:01Z	INFO	[db] Need to update DB
2025-12-20T01:33:01Z	INFO	[db] Downloading DB...	repository="ghcr.io/aquasecurity/trivy-db:2"
19.17 MiB / 78.74 MiB [-------------->______________________________________________] 24.35% ? p/s ?37.69 MiB / 78.74 MiB [----------------------------->_______________________________] 47.87% ? p/s ?57.98 MiB / 78.74 MiB [-------------------------------------------->________________] 73.64% ? p/s ?78.74 MiB / 78.74 MiB [---------------------------------------------->] 100.00% 99.32 MiB p/s ETA 0s78.74 MiB / 78.74 MiB [---------------------------------------------->] 100.00% 99.32 MiB p/s ETA 0s78.74 MiB / 78.74 MiB [---------------------------------------------->] 100.00% 99.32 MiB p/s ETA 0s78.74 MiB / 78.74 MiB [---------------------------------------------->] 100.00% 92.91 MiB p/s ETA 0s78.74 MiB / 78.74 MiB [---------------------------------------------->] 100.00% 92.91 MiB p/s ETA 0s78.74 MiB / 78.74 MiB [----------------------------------

📊 Critical=0, High=4, Medium=17
🤖 Calling Gemini 2.5 Flash via Vertex AI...
🔎 Raw Gemini response:
{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "```json\n{\n  \"decision\": \"DEPLOY\",\n  \"risk_level\": \"HIGH\",\n  \"summary\": \"Hard policy passed, but 4 high and 17 medium vulnerabilities for an internet-facing production workload without runtime security controls indicate high residual risk.\"\n}\n```"
          }
        ]
      },
      "finishReason": "STOP",
      "avgLogprobs": -1.4758592421008694
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 154,
    "candidatesTokenCount": 62,
    "totalTokenCount": 525,
    "trafficType": "ON_DEMAND",
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 154
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 62
      }
    ],
    "thoughtsTokenCount": 309
  },
  "modelV

JSONDecodeError: Expecting value: line 1 column 1 (char 0)